In [1]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import spacy
import numpy as np
from ast import literal_eval
import time


In [2]:
!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 2.9 MB/s eta 0:00:0000:0100:02
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [9]:
nlp = spacy.load("en_core_web_lg")  # This loads the pre-trained word vectors

In [4]:
tmdb_movies = pd.read_csv("../../data/clean/final_movies.csv") #load dataset

/var/folders/mm/b26l1fbn3lgg74ssxs6chhsh0000gn/T/ipykernel_37969/826469839.py:1: DtypeWarning: Columns (0,1,7) have mixed types. Specify dtype option on import or set low_memory=False.
  tmdb_movies = pd.read_csv("../../data/clean/final_movies.csv") #load dataset


In [6]:
tmdb_movies.shape, tmdb_movies.columns

((725816, 9),
 Index(['Unnamed: 0', 'id', 'title', 'overview', 'keywords', 'genres', 'cast',
        'director', 'popularity'],
       dtype='object'))

In [7]:
# Sample target keywords
target_keywords = ["Action","Mystery", 
"Thriller",
"Suspenseful",
"Gripping" ,
"Plot twist"]

In [8]:
target_keywords

['Action', 'Mystery', 'Thriller', 'Suspenseful', 'Gripping', 'Plot twist']

In [10]:
# Calculate word embeddings for target keywords
target_embeddings = np.mean([nlp(keyword).vector for keyword in target_keywords], axis=0)

In [11]:

# Function to calculate embeddings of movie keywords
def calculate_embeddings(movie_keywords):
    if isinstance(movie_keywords, list):
        return np.mean([nlp(keyword).vector for keyword in movie_keywords], axis=0)    
    return None


In [12]:

# Function to calculate cosine similarity between target keywords and movie keywords
def calculate_similarity(embedding):
    #movie_embeddings = np.mean([nlp(keyword).vector for keyword in movie_keywords.split(",")], axis=0)
    if not isinstance(embedding, np.ndarray):
        return 0.0
    similarity = cosine_similarity([target_embeddings], [embedding])[0][0]
    return similarity


In [13]:
tmdb_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 725816 entries, 0 to 725815
Data columns (total 9 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Unnamed: 0  725816 non-null  object 
 1   id          725806 non-null  object 
 2   title       725796 non-null  object 
 3   overview    605521 non-null  object 
 4   keywords    206610 non-null  object 
 5   genres      521777 non-null  object 
 6   cast        492136 non-null  object 
 7   director    210476 non-null  object 
 8   popularity  725796 non-null  float64
dtypes: float64(1), object(8)
memory usage: 49.8+ MB


In [15]:
tmdb_movies_keywords = tmdb_movies[tmdb_movies["keywords"].isna() == False]

In [27]:
tmdb_movies_keywords = tmdb_movies_keywords.reset_index()


In [30]:
del tmdb_movies_keywords['index']


In [31]:
tmdb_movies_keywords.head()

,Unnamed: 0,id,title,overview,keywords,genres,cast,director,popularity
0,0,2.0,Ariel,After the coal mine he works at closes and his...,"['prison', 'underdog', 'helsinki, finland', 'f...","['Drama', 'Comedy', 'Romance', 'Crime']","['Matti Pellonpää', 'Esko Nikkari', 'Susanna H...",['Aki Kaurismäki'],23.170
1,1,3.0,Shadows in Paradise,"Nikander, a rubbish collector and would-be ent...","['helsinki, finland', 'salesclerk', 'garbage']","['Drama', 'Comedy', 'Romance']","['Aki Kaurismäki', 'Kati Outinen', 'Matti Pell...",['Aki Kaurismäki'],35.528
2,2,5.0,Four Rooms,It's Ted the Bellhop's first night on the job....,"['hotel', ""new year's eve"", 'witch', 'bet', 's...",['Comedy'],"['Marisa Tomei', 'Antonio Banderas', 'Quentin ...","['Quentin Tarantino', 'Robert Rodriguez', 'All...",52.481
3,3,6.0,Judgment Night,"Four young friends, while taking a shortcut en...","['drug dealer', 'chicago, illinois', 'escape',...","['Action', 'Crime', 'Thriller']","['Jeremy Piven', 'Stephen Dorff', 'Peter Green...",['Stephen Hopkins'],41.054
4,4,8.0,Life in Loops (A Megacities RMX),Timo Novotny labels his new project an experim...,['megacities'],['Documentary'],NaN,['Timo Novotny'],10.613


In [44]:
tmdb_movies_keywords["keywords"] = tmdb_movies_keywords["keywords"].apply(literal_eval)

In [45]:
# Apply the function to calculate similarity for each movie
start_time = time.time()
embeddings = tmdb_movies_keywords['keywords'].apply(calculate_embeddings)
end_time = time.time()
print(f"Time taken to calculate {len(embeddings)} embeddings was {end_time-start_time}")

Time taken to calculate 206610 embeddings was 6941.017413854599


In [46]:
embeddings

0         [-2.2576869, -0.47951338, -2.29372, 1.1469345,...
1         [-1.1752855, -1.2179189, -0.26367328, 1.306206...
2         [-0.2116311, 0.17154092, -1.4510907, -1.112288...
3         [-2.0025425, 0.6185693, -1.6088126, 0.08380668...
4         [-0.026181, 0.94735, -3.1535, 1.3683, 1.9055, ...
                                ...                        
206605    [-3.3004348, 0.99205005, -2.1723251, -4.606670...
206606    [1.6747501, 0.44393, -1.9986999, -1.7374, -1.1...
206607    [-1.3433976, 0.91446555, -1.0329688, -0.026978...
206608    [0.7474041, 2.8854349, -2.0129917, -1.6594416,...
206609    [0.02367172, 0.4836885, -2.2233353, -0.5476513...
Name: keywords, Length: 206610, dtype: object

In [22]:
# nlp("mafia").similarity(nlp("crime")), nlp_lg("mafia").similarity(nlp_lg("crime"))

In [43]:
# res = []
# for i in range(len(target_keywords)):
#     keywords_sum = 0
#     for j in range(len(tmdb_movies["tmdb_keywords"][1])):
#         keywords_sum += nlp(target_keywords[i]).similarity(nlp(tmdb_movies["tmdb_keywords"][1][0]))
#     res.append(keywords_sum/len(tmdb_movies["tmdb_keywords"][1]))

In [47]:
score = []
for i in range(len(embeddings)):
    score.append((i, calculate_similarity(embeddings[i])))

In [49]:
score.sort(key=lambda x: (x[1],x[0]), reverse=True)

In [50]:
score[0:5]

[(185007, 0.69505614),
 (168551, 0.69187236),
 (199494, 0.686579),
 (177862, 0.686579),
 (176342, 0.686579)]

In [52]:
for x in score[0:10]:
    print(tmdb_movies["title"][x[0]], tmdb_movies["keywords"][x[0]], x[1])

Um Coelho de Verdade ['child'] 0.69505614
Nocturno de amor ['scholarship', 'love triangle', 'piano lessons'] 0.69187236
Tungsten Grey is the Warmest Colour nan 0.686579
People Like Us nan 0.686579
Disaster Preparedness ['woman director'] 0.686579
Arcadia Lost nan 0.686579
Deadly Target ['martial arts', 'arms dealer', 'cop'] 0.68254626
Sudden Thunder ['revenge', 'murder', 'drugs'] 0.6750954
Portals ['woman director'] 0.6721057
Cuba libre nan 0.67196906


In [56]:
tmdb_movies.head()

,Unnamed: 0,id,title,overview,keywords,genres,cast,director,popularity
0,def append_to_file(src,dst):,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,with open(src,"""r"") as s:",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,s.readline(),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,content = s.read(),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,with open (dst,"""a"") as d:",NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
temp = []
for i in range(len(embeddings)):
    temp.append((i, calculate_similarity(embeddings[i])))

In [15]:
temp[5934]

(5934, 0.7153774)